In [1]:
from datetime import datetime
from functools import lru_cache
import sqlite3

import pandas as pd
import pymongo
import requests

In [2]:
conn = sqlite3.connect('database.sqlite')
cur =  conn.cursor()

In [3]:
teams = [r[0] for r in cur.execute("""SELECT DISTINCT AwayTeam FROM Matches
                                    WHERE (Div = "D1" OR Div = "D2")
                                    AND Season = 2011
                                    ORDER BY AwayTeam""").fetchall()]

In [4]:
def team_goals_wins():
    def _fetch(calc, side, team, ftr):
        return cur.execute(f"""SELECT {calc} FROM Matches
                            WHERE {side} = ?
                            AND FTR LIKE "{ftr}"
                            AND (Div = "D1" OR Div = "D2")
                            AND Season = 2011""", (team,)).fetchone()[0]
        
    out = []
    for team in teams:
        away_goals = _fetch("SUM(FTAG)", "AwayTeam", team, "%")
        home_goals = _fetch("SUM(FTHG)", "HomeTeam", team, "%")
        away_wins = _fetch("COUNT(*)", "AwayTeam", team, "A")
        home_wins = _fetch("COUNT(*)", "HomeTeam", team, "H")

        out.append((team, away_goals, home_goals, away_wins, home_wins))
    return out

In [26]:
team_goals_wins_df = pd.DataFrame(team_goals_wins())
team_goals_wins_df.columns = ("Team", "AwayGoals", "HomeGoals", "AwayWins", "HomeWins")
team_goals_wins_df.head()

,Team,AwayGoals,HomeGoals,AwayWins,HomeWins
0,Aachen,15,15,2,4
1,Augsburg,16,20,2,6
2,Bayern Munich,28,49,9,14
3,Bochum,18,23,3,7
4,Braunschweig,16,21,4,6


In [6]:
with open("./DARK_SKY_KEY.txt", "rt") as f:
    D_S_KEY = f.read()
LAT = "52.520008"
LON = "13.404954"

In [13]:
@lru_cache(maxsize=None)
def ds_data(datestr):
    """Return Dark Sky daily data in Berlin on given date."""
    y, m, d = map(int, datestr.split('-'))
    ts = int(datetime(y, m, d, 12).timestamp())
    url = ("https://api.darksky.net/forecast/" 
           f"{D_S_KEY}/{LAT},{LON},{ts}"
           "?exclude=currently,hourly,flags")
    return requests.get(url).json()['daily']['data'][0]

In [16]:
def was_raining(datestr):
    """Return whether there was precipitation in Berlin on given date."""
    data =  ds_data(datestr)
    return data['precipIntensity'] > 0    

In [47]:
weather_performance = []
for team in teams:
    ndry_games = 0
    ndry_losses = 0
    ndry_wins = 0
    nwet_games = 0
    nwet_losses = 0
    nwet_wins = 0
    
    games = cur.execute("""SELECT Date, AwayTeam, HomeTeam, FTR FROM Matches
                    WHERE (AwayTeam = ? OR HomeTeam = ?)
                    AND (Div = "D1" OR Div = "D2")
                    AND Season = 2011""", (team, team)).fetchall()

    for datestr, away_team, home_team, ftr in games:
        was_win = (away_team == team and ftr == 'A' or home_team == team and ftr == 'H')
        was_loss = (away_team == team and ftr == 'H' or home_team == team and ftr == 'A')
        if was_raining(datestr):
            nwet_games += 1
            if was_win:
                nwet_wins += 1
            elif was_loss:
                nwet_losses += 1
        else:
            ndry_games += 1
            if was_win:
                ndry_wins += 1
            elif was_loss:
                ndry_losses += 1
    #print(team, 100 * ndry_wins / ndry_games, 100 * nwet_wins / nwet_games) 
    weather_performance.append((team, 100 * ndry_wins / ndry_games, 100 * nwet_wins / nwet_games))

In [80]:
weather_performance_df = pd.DataFrame(weather_performance, columns = ("Team", "Dry Win Rate (%)", "Wet WIn Rate (%)"))

In [81]:
team_data_df = team_goals_wins_df.merge(weather_performance_df, on='Team')
team_data_df.head()

,Team,AwayGoals,HomeGoals,AwayWins,HomeWins,Dry Win Rate (%),Wet WIn Rate (%)
0,Aachen,15,15,2,4,16.666667,20.000000
1,Augsburg,16,20,2,6,20.000000,33.333333
2,Bayern Munich,28,49,9,14,65.384615,75.000000
3,Bochum,18,23,3,7,32.000000,22.222222
4,Braunschweig,16,21,4,6,23.076923,50.000000


In [30]:
client = pymongo.MongoClient()

In [44]:
#print(client.admin.command("serverStatus"))

In [33]:
db = client.football_delphi

In [58]:
for _, row in team_data_df.iterrows():
    (team_name, away_goals, home_goals, away_wins, home_wins, dry_win_rate, wet_win_rate) = row
    team = {
        'name': team_name,
        'goals': away_goals + home_goals,
        'wins': away_wins + home_wins,
        'dry_win_rate': dry_win_rate,
        'wet_win_rate': wet_win_rate,
    }
    result = db.teams.insert_one(team)
    #print(team_name, result)

In [57]:
len([r[0] for r in cur.execute("""SELECT Date FROM Matches
                                    WHERE (Div = "D1" OR Div = "D2")
                                    AND Season = 2011""").fetchall()])

612

In [59]:
cur.executescript("""
        CREATE TEMP VIEW IF NOT EXISTS Bundesliga_2011 
        AS
        SELECT * FROM Matches
        WHERE (Div = "D1" OR Div = "D2")
        AND Season = 2011;""")

In [76]:
team_name = teams[4]
def foo():
    return cur.execute("""SELECT Date FROM Bundesliga_2011 AS B
                JOIN Teams_in_Matches AS TIM  ON B.Match_ID=TIM.Match_ID
                JOIN Unique_Teams AS UT ON TIM.Unique_Team_ID=UT.Unique_Team_ID 
                WHERE UT.TeamName=?""", (team_name,)).fetchall()

In [77]:
def bar():
    return cur.execute("""SELECT Date FROM Bundesliga_2011 
                    WHERE AwayTeam=? OR HomeTeam=?""", (team_name, team_name)).fetchall()

In [78]:
%timeit foo()

12.3 ms ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
%timeit bar()

3.47 ms ± 52.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
